In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join


In [ ]:
data_directory = "D:\\MLT assigments\\WALMART\\1\\Second round\\prefinal_datasets"

In [ ]:
data_files = [f for f in listdir(data_directory) if isfile(join(data_directory, f))]
data_files = [f for f in data_files if "csv" in f and f.startswith("201")]


In [ ]:
def get_columns_to_drop():
    data = pd.read_csv(data_directory + "/2019-01-01.csv")
    drop_columns = [col for col in data.columns if "normalized" in col]
    return drop_columns     

In [ ]:
columns_to_drop = get_columns_to_drop()

In [ ]:
# Store all the dataframes into a list 
all_dataframes = []

In [ ]:
for file_name in data_files:
    data = pd.read_csv(data_directory + "/" + file_name)
    data.drop(columns_to_drop, axis=1, inplace=True)
    all_dataframes.append(data.copy())
    print("{} added to the list of dataframes".format(file_name))

In [ ]:
# generate a master data frame to have all the data
main_dataframe = pd.concat(all_dataframes)

In [ ]:
models_count = main_dataframe["model"].value_counts().to_frame()
models_count.columns = ["count"]
models_count['model_name'] = models_count.index
models_count.reset_index(drop=True)

In [ ]:
failure_counts = main_dataframe[main_dataframe["failure"]==1].groupby(["model"]).agg({'model':'count'})
failure_counts.columns = ["failed_count"]
failure_counts['model_name'] = failure_counts.index
failure_counts = failure_counts.reset_index(drop=True)

In [ ]:

no_failure_counts = main_dataframe[main_dataframe["failure"]==0].groupby(["model"]).agg({'model':'count'})
no_failure_counts.columns = ["no_failed_count"]
no_failure_counts['name'] = no_failure_counts.index
no_failure_counts = no_failure_counts.reset_index(drop=True)
no_failure_counts.shape

In [ ]:
merged_data_1 = pd.merge(no_failure_counts, failure_counts, left_on='model_name', right_on='model_name', how="left")


In [ ]:
merged_data_2 = pd.merge(models_count, merged_data_1, left_on='model_name', right_on='model_name', how="left")
merged_data_2 = merged_data_2.fillna(0)
merged_data_2["failed_count"] = merged_data_2["failed_count"].astype('int32')
merged_data_2 = merged_data_2.sort_values(by='failed_count', ascending=False)
merged_data_2.to_csv(r'overall_stats.csv', index = False)


In [ ]:
model_name = "ST12000NM0007"
model_1_data = main_dataframe[main_dataframe["model"] == model_name]


In [ ]:
model_1_data = model_1_data.dropna(axis=1, how='all') 

In [ ]:
numerical_cols = model_1_data.select_dtypes([np.number]).columns
std_deviation = model_1_data[numerical_cols].std()
cols_to_drop = std_deviation[std_deviation == 0].index

In [ ]:

model_1_data = model_1_data.dropna()


In [ ]:
unique_value_cols = model_1_data.columns[model_1_data.nunique() <= 1].tolist()
unique_value_cols.remove('model')
unique_value_cols.remove('capacity_bytes')
model_1_data = model_1_data.drop(unique_value_cols, axis=1)

In [ ]:
model_1_data = model_1_data.sort_values(['serial_number', 'date'], ascending=[True, True])


In [ ]:
def clean_data(model_name):
    model_data = main_dataframe[main_dataframe["model"] == model_name]
    model_data = model_data.dropna(axis=1, how='all')
    model_data = model_data.dropna()
    unique_value_cols = model_data.columns[model_data.nunique() <= 1].tolist()
    unique_value_cols.remove('model')
    unique_value_cols.remove('capacity_bytes')
    model_data = model_data.drop(unique_value_cols, axis=1)
    model_data = model_data.sort_values(['serial_number', 'date'], ascending=[True, True])
    return model_data

In [ ]:
cleaned_model_data = clean_data(model_name)

In [ ]:
export_filename = model_name+"_finalcleaned.csv"
cleaned_model_data.to_csv(export_filename, index = False)

In [ ]:
last_day_data = cleaned_model_data.groupby('serial_number').tail(1)
export_filename = model_name+"last.csv"
last_day_data.to_csv(export_filename, index = False)

In [ ]:
n = 10 # Number of days data needed
last_n_days_data = cleaned_model_data.groupby('serial_number').tail(n)
for index, row in last_day_data.iterrows():
    if row["failure"] == 1:
        last_n_days_data["failure"].mask(last_n_days_data["serial_number"] == row["serial_number"], 1, inplace=True)
export_filename = model_name+"_last_"+ str(n) + "_days.csv"
last_n_days_data.to_csv(export_filename, index = False)

In [ ]:
def get_columns_to_drop_normalised():
    data = pd.read_csv(data_directory +"/2019-01-01.csv")
    drop_columns = [col for col in data.columns if "raw" in col]
    return drop_columns 

columns_to_drop_normalised = get_columns_to_drop_normalised()

In [ ]:
all_dataframes = []
for file_name in data_files:
    data = pd.read_csv(data_directory + "/" +file_name)
    data.drop(columns_to_drop_normalised, axis = 1, inplace=True)
    all_dataframes.append(data.copy())

In [ ]:
model_name = "ST12000NM0007"
def clean_data_normalised(model_name):
    model_data = main_dataframe[main_dataframe["model"] == model_name]
    model_data = model_data.dropna(axis=1, how='all')
    model_data = model_data.dropna()
    unique_value_cols = model_data.columns[model_data.nunique() <= 1].tolist()
    unique_value_cols.remove('model')
    unique_value_cols.remove('capacity_bytes')
    model_data = model_data.drop(unique_value_cols, axis=1)
    model_data = model_data.sort_values(['serial_number', 'date'], ascending=[True, True])
    return model_data

cleaned_normalised_data = clean_data_normalised(model_name)

last_day_normalised_data = cleaned_normalised_data.groupby('serial_number').tail(1)
export_filename = model_name+"_lastnorm.csv"
last_day_normalised_data.to_csv(export_filename, index = False)

n = 10 # Number of days data needed
last_n_days_normalised_data =  cleaned_normalised_data.groupby('serial_number').tail(n)
for index, row in last_day_normalised_data.iterrows():
    if row["failure"] == 1:
        last_n_days_normalised_data["failure"].mask(last_n_days_normalised_data["serial_number"] == row["serial_number"], 1, inplace=True)
export_filename = model_name+"_last_"+ str(n) + "_days_normalised.csv"
last_n_days_normalised_data.to_csv(export_filename, index = False)